<a href="https://colab.research.google.com/github/bhave068/Devanagiri-Recognizer/blob/master/Facial%20key%20point%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from time import sleep
import os

In [0]:
train_data=pd.read_csv("/content/drive/My Drive/Facial keypoint detection/training.csv")
test_data=pd.read_csv("/content/drive/My Drive/Facial keypoint detection/test.csv")
lookid_data=pd.read_csv("/content/drive/My Drive/Facial keypoint detection/IdLookupTable.csv")

In [0]:
train_data.head().T

In [0]:
train_data.isnull().any().value_counts()

In [0]:
train_data.fillna(method='ffill',inplace=True)

In [0]:
train_data.isnull().any().value_counts()

In [0]:
train_data.shape

In [0]:
imag=[];
for i in range(0,7049):
  img=train_data['Image'][i].split(' ')
  img=['0' if x==''else x for x in img]
  imag.append(img)

In [0]:
image_list = np.array(imag,dtype = 'uint8')
X_train = image_list.reshape(-1,96,96,1)

In [0]:
del imag

In [0]:
plt.imshow(X_train[5].reshape(96,96),cmap='gray')

In [0]:
training=train_data.drop('Image',axis=1)
y_train = []
for i in range(0,7049):
    y = training.iloc[i,:]
    y_train.append(y)
y_train=np.array(y_train,dtype='uint8')

In [0]:
y_train.shape

In [0]:
X_train.shape

In [0]:
from keras.layers import Conv2D,Dropout,Dense,Flatten
from keras.models import Sequential

In [0]:
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [0]:
model=Sequential()

In [0]:
model.add(Convolution2D(32,(3,3),padding='same',use_bias=False,input_shape=(96,96,1)))

In [0]:
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

In [0]:
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

In [0]:
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

In [0]:
model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

In [0]:
model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

In [0]:
model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

In [0]:
model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


In [0]:
model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

In [0]:
model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

In [0]:
model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(512,activation='relu'))

In [0]:
model.add(Dropout(0.1))

In [0]:
model.add(Dense(30))

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [0]:
model.fit(X_train,y_train,epochs=50,batch_size=256,validation_split=0.2)

In [0]:
timag = []
for i in range(0,1783):
    timg = test_data['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]
    
    timag.append(timg)

In [0]:
timage_list = np.array(timag,dtype = 'uint8')
X_test = timage_list.reshape(-1,96,96,1) 

In [0]:
del timag

In [0]:
plt.imshow(X_test[20].reshape(96,96),cmap = 'gray')
plt.show()

In [0]:
pred = model.predict(X_test)

In [0]:
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)

In [0]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [0]:
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))

In [0]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

In [0]:
rowid = pd.Series(rowid,name = 'RowId')

In [0]:
loc = pd.Series(preded,name = 'Location')


In [0]:
submission = pd.concat([rowid,loc],axis = 1)

In [0]:
submission.to_csv('face_key_detection_submission.csv',index = False)